In [1]:
import pandas as pd
import numpy as np
import os
import difflib

os.chdir('../DrugCell/')

In [13]:
# take a subset.
cell2ind = pd.read_csv('data/cell2ind.txt', header=None, sep='\t')
# take a whole things
gene2ind = pd.read_csv('data/gene2ind.txt', header=None, sep='\t')
# take a subset.
drug2ind = pd.read_csv('data/drug2ind.txt', header=None, sep='\t')
# take a subset.
cell2mutation = pd.read_csv('data/cell2mutation.txt', header=None, sep=',')
# take a subset.
drug2fingerprint = pd.read_csv('data/drug2fingerprint.txt', header=None, sep=',')
# take a whole things
drugcell_ont = pd.read_csv('data/drugcell_ont.txt', header=None, sep='\t')
# make train, test and validation
rcellminer = pd.read_csv('data/rcellminer_test.txt', header=None, sep='\t')

# train = pd.read_csv('data/drugcell_train.txt', header=None, sep='\t')
# test = pd.read_csv('data/drugcell_test.txt', header=None, sep='\t')



In [58]:
drug2ind

,0,1
0,0,C[C@H]1CN(C(=O)C2=C(C(=CC=C2)NS(=O)(=O)C3=CC=C...
1,1,C1C2CC3CC1CC(C2)(C3)C4=C(C=C5C=C(C=CC5=C4)C6=C...
2,2,CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)N/C(=C\3/C4=...
3,3,CC(CS(=O)(=O)C1=CC=C(C=C1)F)(C(=O)NC2=CC(=C(C=...
4,4,C[C@H]1CN(C(=O)C2=C(C=CC(=C2)NS(=O)(=O)C3=CN(C...
...,...,...
679,679,CC1=CC(=CC2=C1NC(=C3C(=CC=NC3=O)NC[C@H](C4=CC(...
680,680,CN(C)CC1CN(C1)C(=O)NC2=C(C=CC(=C2)C3=CC=C(C=C3...
681,681,C1CN(CCC1CCCCNC(=O)/C=C/C2=CN=CC=C2)C(=O)C3=CC...
682,682,CC(C)(C)C1=CC=C(C=C1)S(=O)(=O)NCC2=CC=C(C=C2)C...


In [44]:
drugcell_ont.head()

,0,1,2
0,GO:0007005,GO:0007006,default
1,GO:0007005,GO:0008637,default
2,GO:0006281,GO:0006284,default
3,GO:0006281,GO:0006283,default
4,GO:0006281,GO:0019985,default


In [52]:
gene2ind.head()
gene2ind.shape

(3008, 2)

In [47]:
cell2ind.head()

,0,1
0,0,NCIH810_LUNG
1,1,HN_UPPER_AERODIGESTIVE_TRACT
2,2,COV434_OVARY
3,3,CCRFCEM_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE
4,4,PECAPJ49_UPPER_AERODIGESTIVE_TRACT


In [48]:
drug2ind.head()

,0,1
0,0,C[C@H]1CN(C(=O)C2=C(C(=CC=C2)NS(=O)(=O)C3=CC=C...
1,1,C1C2CC3CC1CC(C2)(C3)C4=C(C=C5C=C(C=CC5=C4)C6=C...
2,2,CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)N/C(=C\3/C4=...
3,3,CC(CS(=O)(=O)C1=CC=C(C=C1)F)(C(=O)NC2=CC(=C(C=...
4,4,C[C@H]1CN(C(=O)C2=C(C=CC(=C2)NS(=O)(=O)C3=CN(C...


In [51]:
cell2mutation

,0,1,2,3,4,5,6,7,8,9,...,2998,2999,3000,3001,3002,3003,3004,3005,3006,3007
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1220,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1221,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1222,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1223,0,0,0,1,1,0,1,1,1,1,...,0,1,1,0,0,0,0,0,0,1


In [54]:
drug2fingerprint.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
train

,0,1,2
0,KYSE450_OESOPHAGUS,C1CC(=O)NC(=O)C1N2C(=O)C3=CC=CC=C3C2=O,1.006587
1,SUDHL1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,C1CN(CCN1)C(=O)C2=CC=C(C=C2)/C=C/C3=NNC4=CC=CC...,0.705210
2,OCIAML3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,COC1=C(C=C2C(=C1)N=CN=C2NC3=CC(=C(C=C3)F)Cl)OC...,0.885490
3,SCC9_UPPER_AERODIGESTIVE_TRACT,C1=CC=C2C(=C1)C3=CC=CC=C3N2CCC4=NC5=CC=CC=C5N4,0.955772
4,SNU489_CENTRAL_NERVOUS_SYSTEM,CCCCCCCCCCC(C)(C)C(=O)NC1=C(C=C(C=C1OC)OC)OC,1.012310
...,...,...,...
9995,MC116_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,CNC(=O)C1=NC=CC(=C1)OC2=CC=C(C=C2)NC(=O)NC3=CC...,0.965572
9996,LS123_LARGE_INTESTINE,C[C@]12CC[C@@]3(CCC(C[C@@H]3[C@@H]1C(=O)C=C4[C...,0.777269
9997,BE13_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,CC1=NC=C(C=C1)OC2=C(C=C(C=C2)NC3=NC=NC4=C3C=C(...,0.924181
9998,KMH2_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,CCOC(=O)C1=C(SC2=C1CC(NC2(C)C)(C)C)NC(=O)C3=CC...,0.953433


In [56]:
test

,0,1,2
0,LS123_LARGE_INTESTINE,CNC(=O)C1=NC=CC(=C1)OC2=CC(=C(C=C2)NC(=O)NC3=C...,0.894224
1,RT4_URINARY_TRACT,C[C@@H](C(=O)N[C@H]1C2=CC=CC=C2CCN(C1=O)C)NC(=...,0.874261
2,SCABER_URINARY_TRACT,C1CCN(C1)C(=O)NC2=CC=CC(=C2)NC3=NC=C(C(=N3)NCC...,0.960072
3,TE617T_SOFT_TISSUE,C1CN(CCC12C(=O)NCN2C3=CC=CC=C3)CCNC(=O)C4=CC5=...,0.987354
4,MOGGCCM_CENTRAL_NERVOUS_SYSTEM,CC1=CN=C(N=C1NC2=CC(=CC=C2)S(=O)(=O)NC(C)(C)C)...,0.871997
...,...,...,...
995,L1236_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,CC[C@@H]1C(=O)N(C2=CN=C(N=C2N1C3CCCC3)NC4=C(C=...,0.822487
996,JK1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,CCC1=CC=CC=C1NC(=O)CSC(=O)NNC(=O)[C@@H](CC2=CN...,0.829568
997,A427_LUNG,CCN1C=C(C(=N1)C2=CC=C(C=C2)NC(=O)N(C)C)C3=C4C=...,0.860422
998,ETK1_BILIARY_TRACT,CC1=C(C=C(C=C1)C(=O)NC2=CC(=CC(=C2)N3C=C(N=C3)...,0.951400


In [14]:
len(set(drugcell_ont[drugcell_ont[2] == 'gene'][1]))

3008

In [43]:
pd.DataFrame(sorted(set(train[0]))).to_csv('/Users/yoshitakainoue/Downloads/train_sample.csv')

In [18]:
print('Cell Index:' + str(cell2ind.shape))
print('Gene Index:' + str(gene2ind.shape))
print('Drug Index:' + str(drug2ind.shape))
print('Cell Mutation:' + str(cell2mutation.shape))
print('Drug Fingerprint:' + str(drug2fingerprint.shape))
print('Ontology: ' + str(drugcell_ont.shape))

Cell Index:(1225, 2)
Gene Index:(3008, 2)
Drug Index:(684, 2)
Cell Mutation:(1225, 3008)
Drug Fingerprint:(684, 2048)
Ontology: (62920, 3)


In [33]:
cell2ind

,0,1
0,0,NCIH810_LUNG
1,1,HN_UPPER_AERODIGESTIVE_TRACT
2,2,COV434_OVARY
3,3,CCRFCEM_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE
4,4,PECAPJ49_UPPER_AERODIGESTIVE_TRACT
...,...,...
1220,1220,IHH4_THYROID
1221,1221,TE11_OESOPHAGUS
1222,1222,C3A_LIVER
1223,1223,HCT15_LARGE_INTESTINE


In [34]:
gene2ind

,0,1
0,0,AADAC
1,1,AADAT
2,2,AAK1
3,3,AARS
4,4,AASDHPPT
...,...,...
3003,3003,YWHAH
3004,3004,YWHAQ
3005,3005,ZAP70
3006,3006,ZMPSTE24


In [35]:
drug2ind

,0,1
0,0,C[C@H]1CN(C(=O)C2=C(C(=CC=C2)NS(=O)(=O)C3=CC=C...
1,1,C1C2CC3CC1CC(C2)(C3)C4=C(C=C5C=C(C=CC5=C4)C6=C...
2,2,CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)N/C(=C\3/C4=...
3,3,CC(CS(=O)(=O)C1=CC=C(C=C1)F)(C(=O)NC2=CC(=C(C=...
4,4,C[C@H]1CN(C(=O)C2=C(C=CC(=C2)NS(=O)(=O)C3=CN(C...
...,...,...
679,679,CC1=CC(=CC2=C1NC(=C3C(=CC=NC3=O)NC[C@H](C4=CC(...
680,680,CN(C)CC1CN(C1)C(=O)NC2=C(C=CC(=C2)C3=CC=C(C=C3...
681,681,C1CN(CCC1CCCCNC(=O)/C=C/C2=CN=CC=C2)C(=O)C3=CC...
682,682,CC(C)(C)C1=CC=C(C=C1)S(=O)(=O)NCC2=CC=C(C=C2)C...


In [36]:
cell2mutation

,0,1,2,3,4,5,6,7,8,9,...,2998,2999,3000,3001,3002,3003,3004,3005,3006,3007
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1220,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1221,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1222,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1223,0,0,0,1,1,0,1,1,1,1,...,0,1,1,0,0,0,0,0,0,1


In [32]:
drugcell_ont

,0,1,2
0,GO:0007005,GO:0007006,default
1,GO:0007005,GO:0008637,default
2,GO:0006281,GO:0006284,default
3,GO:0006281,GO:0006283,default
4,GO:0006281,GO:0019985,default
...,...,...,...
62915,GO:2001240,BCL2L1,gene
62916,GO:2001240,AKT1,gene
62917,GO:0008150,LYPLAL1,gene
62918,GO:0008150,ABHD11,gene


# Shape

In [3]:
print('Cell Index:' + str(cell2ind.shape))
print('Gene Index:' + str(gene2ind.shape))
print('Drug Index:' + str(drug2ind.shape))
print('Cell Mutation:' + str(cell2mutation.shape))
print('Drug Fingerprint:' + str(drug2fingerprint.shape))

Cell Index:(1225, 2)
Gene Index:(3008, 2)
Drug Index:(684, 2)
Cell Mutation:(1225, 3008)
Drug Fingerprint:(684, 2048)


# Sample data

In [10]:
test.head()
# CELL LINE : SMILES

,0,1,2
0,LS123_LARGE_INTESTINE,CNC(=O)C1=NC=CC(=C1)OC2=CC(=C(C=C2)NC(=O)NC3=C...,0.894224
1,RT4_URINARY_TRACT,C[C@@H](C(=O)N[C@H]1C2=CC=CC=C2CCN(C1=O)C)NC(=...,0.874261
2,SCABER_URINARY_TRACT,C1CCN(C1)C(=O)NC2=CC=CC(=C2)NC3=NC=C(C(=N3)NCC...,0.960072
3,TE617T_SOFT_TISSUE,C1CN(CCC12C(=O)NCN2C3=CC=CC=C3)CCNC(=O)C4=CC5=...,0.987354
4,MOGGCCM_CENTRAL_NERVOUS_SYSTEM,CC1=CN=C(N=C1NC2=CC(=CC=C2)S(=O)(=O)NC(C)(C)C)...,0.871997


In [11]:
train[train[0] == 'KYSE450_OESOPHAGUS']

,0,1,2
0,KYSE450_OESOPHAGUS,C1CC(=O)NC(=O)C1N2C(=O)C3=CC=CC=C3C2=O,1.006587
67,KYSE450_OESOPHAGUS,COC1=C(C=CC(=C1)C=CC(=O)CC(=O)C=CC2=CC(=C(C=C2...,0.890410
382,KYSE450_OESOPHAGUS,CCCS(=O)(=O)NC1=C(C(=C(C=C1)F)C(=O)C2=CNC3=NC=...,1.001322
768,KYSE450_OESOPHAGUS,CCOC(=O)C1=CC=C(C=C1)N2C(=O)C(=CC3=CC=C(O3)[N+...,0.976716
785,KYSE450_OESOPHAGUS,CC1=CC2=C(C=C1)N=C(C3=NC=C(N23)C)NCCN,0.772537
2173,KYSE450_OESOPHAGUS,C1=C(C(=O)NC(=O)N1)F,0.481351
2266,KYSE450_OESOPHAGUS,COC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=C(C=C3)NC(=O)...,0.982268
2885,KYSE450_OESOPHAGUS,CN1CCN(CC1)CCCN2C3=CC=CC=C3SC4=C2C=C(C=C4)Cl,1.013487
3538,KYSE450_OESOPHAGUS,CC(C)OC(=O)C1=CN(CC(C2=C1NC3=CC=CC=C32)(C)C)C(...,0.976186
3572,KYSE450_OESOPHAGUS,COC(=O)CNC(=O)C(=O)OC,0.734794


In [12]:
test[test[0] == 'KYSE450_OESOPHAGUS']

,0,1,2
372,KYSE450_OESOPHAGUS,COC1=CC(=C(C=C1)OC)C2=NN=C3N2N=C(CS3)C4=CC(=C(...,0.782034


# Shape

In [13]:
print('Cell Index:' + str(cell2ind.shape))
print('Gene Index:' + str(gene2ind.shape))
print('Drug Index:' + str(drug2ind.shape))
print('Cell Mutation:' + str(cell2mutation.shape))
print('Drug Fingerprint:' + str(drug2fingerprint.shape))
print('Gene Ontology:' + str(drugcell_ont.shape))

Cell Index:(1225, 2)
Gene Index:(3008, 2)
Drug Index:(684, 2)
Cell Mutation:(1225, 3008)
Drug Fingerprint:(684, 2048)
Gene Ontology:(62920, 3)


In [14]:
cell2ind.head()
# index : Cell Line

,0,1
0,0,NCIH810_LUNG
1,1,HN_UPPER_AERODIGESTIVE_TRACT
2,2,COV434_OVARY
3,3,CCRFCEM_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE
4,4,PECAPJ49_UPPER_AERODIGESTIVE_TRACT


In [15]:
gene2ind.head()
# index : Gene Name

,0,1
0,0,AADAC
1,1,AADAT
2,2,AAK1
3,3,AARS
4,4,AASDHPPT


In [26]:
drug2ind.head()
# index : SMILES

,0,1
0,0,C[C@H]1CN(C(=O)C2=C(C(=CC=C2)NS(=O)(=O)C3=CC=C...
1,1,C1C2CC3CC1CC(C2)(C3)C4=C(C=C5C=C(C=CC5=C4)C6=C...
2,2,CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)N/C(=C\3/C4=...
3,3,CC(CS(=O)(=O)C1=CC=C(C=C1)F)(C(=O)NC2=CC(=C(C=...
4,4,C[C@H]1CN(C(=O)C2=C(C=CC(=C2)NS(=O)(=O)C3=CN(C...


In [17]:
cell2mutation.head()
# Mutation for each Cell Line
# columns = Gene
# index = cell line
# Probably I can use ↓ Relating Gene Alteration Patterns to Drug Responses
# https://www.bioconductor.org/packages/release/bioc/vignettes/rcellminer/inst/doc/rcellminerUsage.html
# https://rdrr.io/bioc/rcellminer/man/getBinaryMutationData.html

,0,1,2,3,4,5,6,7,8,9,...,2998,2999,3000,3001,3002,3003,3004,3005,3006,3007
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
drug2fingerprint.head()
# Morgan Fingerprint for each drug

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
